## A Gradio Interface to Compare Diverse CNN Architectures.
This notebook is part of Module 3 of the [ODSC Europe 2021](https://odsc.com/europe/) workshop [Adversarial Attacks and Defence in Computer Vision 101](https://odsc.com/speakers/adversarial-attacks-and-defence-in-computer-vision-101/).  
![ODSC Logo](https://opendatascience.com/wp-content/uploads/2021/01/odsceutop.png)  
It includes a [Gradio](https://github.com/gradio-app/gradio-UI) interface to compare diverse CNN models performance in image classification tasks.  
This notebook can be executed in a CPU runtime with good performance. No need to connect it to a GPU or TPU runtime.

Install the Gradio Python package from PyPi.

In [1]:
!pip install -q gradio 

     |████████████████████████████████| 1.1MB 6.9MB/s 
     |████████████████████████████████| 1.9MB 18.7MB/s 
     |████████████████████████████████| 215kB 37.3MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 962kB 38.5MB/s 
     |████████████████████████████████| 3.2MB 38.4MB/s 


Import the necessary dependencies.

In [2]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests

Download the human-readable labels for the ImageNet data set.

In [3]:
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

Load the models.

In [4]:
inception_net = tf.keras.applications.InceptionV3(weights='imagenet')
efficient_net = tf.keras.applications.EfficientNetB5(weights='imagenet')
mobile_net = tf.keras.applications.MobileNetV2(weights='imagenet')

14540800/14536120 [==============================] - 0s 0us/step


Define the classification function for each model.

In [5]:

def classify_image_with_inception_net(im):
	im = Image.fromarray(im.astype('uint8'), 'RGB')
	im = im.resize((299, 299))
	arr = np.array(im).reshape((-1, 299, 299, 3))
	arr = tf.keras.applications.inception_v3.preprocess_input(arr)
	prediction = inception_net.predict(arr).flatten()
 
	return {labels[i]: float(prediction[i]) for i in range(1000)}

def classifiy_image_with_efficient_net(im):
  im = Image.fromarray(im.astype('uint8'), 'RGB')
  im = im.resize((456, 456))
  arr = np.array(im).reshape((-1, 456, 456, 3))
  arr = tf.keras.applications.efficientnet.preprocess_input(arr)
  prediction = efficient_net.predict(arr).flatten()

  return {labels[i]: float(prediction[i]) for i in range(1000)}

def classify_image_with_mobile_net(im):
  im = Image.fromarray(im.astype('uint8'), 'RGB')
  im = im.resize((224, 224))
  arr = np.array(im).reshape((-1, 224, 224, 3))	
  arr = tf.keras.applications.mobilenet.preprocess_input(arr)
  prediction = mobile_net.predict(arr).flatten()
  
  return {labels[i]: float(prediction[i]) for i in range(1000)}

Build the Gradio interface.

In [6]:
imagein = gr.inputs.Image()
label = gr.outputs.Label(num_top_classes=3)
debug = False

gr.Interface(
    [classify_image_with_inception_net, classifiy_image_with_efficient_net, classify_image_with_mobile_net], 
    imagein, 
    label,
    title="CNN Models Comparison",
    description="Let's compare some state-of-the-art machine learning models that classify images into one of 1,000 categories."
).launch(debug=debug);

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://23982.gradio.app
Interface loading below...


Tip: Add interpretation to your model by simply adding `interpretation="default"` to `Interface()`


#### The interface is now live on the gradio.app link shown above. Go ahead and open that in a new tab!

Please contact us [here](mailto:team@gradio.app) if you have any questions, or [open an issue](https://github.com/gradio-app/gradio-UI/issues/new/choose) at our github repo.

